In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
ds=pd.read_csv("/content/fakeReviewData.csv")

In [ ]:
ds.columns

Index(['category', 'rating', 'label', 'text_'], dtype='object')

In [ ]:
ds["category"].value_counts()
ds["rating"].value_counts()
ds['label'].value_counts()


,count
label,
CG,20216
OR,20216


In [ ]:
ds.pivot_table(columns="category", index="label", aggfunc="size", fill_value=0)

category,Books_5,Clothing_Shoes_and_Jewelry_5,Electronics_5,Home_and_Kitchen_5,Kindle_Store_5,Movies_and_TV_5,Pet_Supplies_5,Sports_and_Outdoors_5,Tools_and_Home_Improvement_5,Toys_and_Games_5
label,,,,,,,,,,
CG,2185,1924,1994,2028,2365,1794,2127,1973,1929,1897
OR,2185,1924,1994,2028,2365,1794,2127,1973,1929,1897


In [ ]:
ds.pivot_table(index="rating", columns="label", aggfunc="size", fill_value=0)

label,CG,OR
rating,,
1.0,1063,1092
2.0,962,1005
3.0,1952,1834
4.0,3920,4045
5.0,12319,12240


In [ ]:
ds["text_"] = ds["text_"].str.replace(r'\d+', "", regex=True).str.lower()
ds["text_"] = ds["text_"].str.translate(str.maketrans('', '', string.punctuation)).str.strip().str.replace(r'\s+',' ', regex=True)

In [ ]:
ds["text_"] = ds["text_"].apply(lambda x: " ".join(word for word in word_tokenize(x) if word not in stopwords))

In [ ]:
lemmatizer = WordNetLemmatizer()
ds["text_"] = ds["text_"].apply(lambda x : "".join(lemmatizer.lemmatize(word) for word in x))

In [ ]:
tfidf_vectorizer=TfidfVectorizer(max_features=500)
tfidf_vector = tfidf_vectorizer.fit_transform(ds["text_"])

In [ ]:
dfidf_df=pd.DataFrame(tfidf_vector.toarray(),columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
dfidf_df.memory_usage(deep=True).sum()

161728128

In [ ]:
ds=pd.concat([ds,dfidf_df],axis=1)

In [ ]:
ds

,category,rating,label,text_,able,absolutely,acting,action,actually,add,...,would,writing,written,wrong,year,years,yet,youll,young,youre
0,Home_and_Kitchen_5,5.0,CG,love well made sturdy comfortable love itvery ...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,Home_and_Kitchen_5,5.0,CG,love great upgrade original ive mine couple years,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.429135,0.0,0.0,0.0,0.000000
2,Home_and_Kitchen_5,5.0,CG,pillow saved back love look feel pillow,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,Home_and_Kitchen_5,1.0,CG,missing information use great product price,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Home_and_Kitchen_5,5.0,CG,nice set good quality set two months,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,read reviews saying bra ran small ordered two ...,0.172311,0.0,0.0,0.0,0.094095,0.0,...,0.051699,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,wasnt sure exactly would little large small si...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.205517,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,wear hood wear hood wear jacket without hood s...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.051273,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.291747
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,liked nothing dress reason gave stars ordered ...,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.039123,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
